In [1]:
import wget
import os.path as osp

# Check if disprot data is already downloaded:
if not osp.exists('disprot.tsv'):
    wget.download("https://disprot.org/api/search?release=2022_03&show_ambiguous=true&show_obsolete=false&format=tsv&namespace=all&get_consensus=false", "disprot.tsv")

In [2]:
import pandas as pd
from aaindex.aaindex import aaindex

disprot = pd.read_csv('disprot.tsv', sep='\t', header=0)
# Drop duplicates:
disprot = disprot.drop_duplicates(subset=['acc', 'start', 'end'])
disprot

,acc,name,organism,ncbi_taxon_id,disprot_id,region_id,start,end,term_namespace,term,ec,reference,region_sequence,confidence,obsolete
0,P03265,DNA-binding protein,Human adenovirus C serotype 5,28285,DP00003,DP00003r002,294,334,Structural state,IDPO:00076,ECO:0006220,pmid:8632448,EHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNT,NaN,NaN
1,P03265,DNA-binding protein,Human adenovirus C serotype 5,28285,DP00003,DP00003r004,454,464,Structural state,IDPO:00076,ECO:0006220,pmid:8632448,VYRNSRAQGGG,NaN,NaN
2,P49913,Cathelicidin antimicrobial peptide,Homo sapiens,9606,DP00004,DP00004r001,134,170,Structural state,IDPO:00076,ECO:0006206,pmid:9452503,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,NaN,NaN
5,P03045,Antitermination protein N,Escherichia phage lambda,10710,DP00005,DP00005r001,1,107,Structural state,IDPO:00076,ECO:0006165,pmid:9659923,MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKS...,NaN,NaN
14,P03045,Antitermination protein N,Escherichia phage lambda,10710,DP00005,DP00005r012,1,22,Structural transition,IDPO:00050,ECO:0006165,pmid:9659923,MDAQTRRRERRAEKQAQWKAAN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10007,A0A2Z5UJ33,Nucleoprotein,Influenza A virus,382835,DP03573,DP03573r005,73,91,Structural state,IDPO:00076,ECO:0006220,pmid:17151603,ERRNKYLEEHPSAGKDPKK,NaN,NaN
10008,A0A2Z5UJ33,Nucleoprotein,Influenza A virus,382835,DP03573,DP03573r006,203,212,Structural state,IDPO:00076,ECO:0006220,pmid:17151603,DRNFWRGENG,NaN,NaN
10010,P03496,Non-structural protein 1,Influenza A virus (strain A/Puerto Rico/8/1934...,211044,DP03575,DP03575r001,204,230,Structural state,IDPO:00076,ECO:0006220,pmid:21464929,RSSNENGRPPLTPKQKREMAGTIRSEV,NaN,NaN
10011,P03496,Non-structural protein 1,Influenza A virus (strain A/Puerto Rico/8/1934...,211044,DP03575,DP03575r002,73,82,Structural state,IDPO:00076,ECO:0006220,pmid:20133840,SDEALKMTMA,NaN,NaN


In [3]:
import requests
import time

def full_seq_from_uniprot(uniprot_id):
    url = 'https://www.uniprot.org/uniprot/' + uniprot_id + '.fasta'
    fasta = requests.get(url).text
    #time.sleep(1)
    return "".join(fasta.split('\n')[1:])

asc2seq = {}
for acc in disprot['acc'].unique():
    asc2seq[acc] = full_seq_from_uniprot(acc)

len(asc2seq)

2365

In [4]:
def get_non_disordered_data(seq, regions):
    for region in sorted(regions, key=lambda x: x[0], reverse=True):
        seq = seq[:region[0]] + seq[region[1]:]
    return seq

asc2non_disordered_seq = {}
for acc in asc2seq:
    start_pos = disprot.loc[disprot['acc'] == acc, 'start'].values
    end_pos = disprot.loc[disprot['acc'] == acc, 'end'].values
    asc2non_disordered_seq[acc] = get_non_disordered_data(asc2seq[acc], list(zip(start_pos, end_pos)))

len(asc2non_disordered_seq)

2365

In [8]:
import numpy as np

aa_index_feats = aaindex.record_codes()

def get_avg_feats_per_sequence(seq):
    features = np.zeros(len(aa_index_feats))
    for i, feat in enumerate(aa_index_feats):
        feat_vals = aaindex[feat]['values']
        features[i] = np.average(np.array([feat_vals[aa] for aa in seq.replace('X', '').replace('U', '').replace('Z', '')]))
    return features

prepared_data = []
for asc, ordered_seq in asc2non_disordered_seq.items():
    prepared_data.append([asc, 0, disprot.loc[disprot['acc'] == asc, 'ncbi_taxon_id'].values[0], len(ordered_seq)] + list(get_avg_feats_per_sequence(ordered_seq)))

for i, row in disprot.iterrows():
    prepared_data.append([row['acc'], 1, row['ncbi_taxon_id'], len(row['region_sequence'])] + list(get_avg_feats_per_sequence(row['region_sequence'])))

prepared_data = pd.DataFrame(prepared_data, columns=['asc', 'is_disordered', 'taxon', 'seq_length']+[f'{feat}_avg' for feat in aa_index_feats])

prepared_data.head()

C:\Users\austi\anaconda3\envs\IDP-Predictor\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\austi\anaconda3\envs\IDP-Predictor\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,asc,is_disordered,taxon,seq_length,ANDN920101_avg,ARGP820101_avg,ARGP820102_avg,ARGP820103_avg,AURR980101_avg,AURR980102_avg,...,YUTK870104_avg,ZASB820101_avg,ZHOH040101_avg,ZHOH040102_avg,ZHOH040103_avg,ZIMJ680101_avg,ZIMJ680102_avg,ZIMJ680103_avg,ZIMJ680104_avg,ZIMJ680105_avg
0,P03265,0,28285,479,4.377641,0.904342,0.970501,1.018914,1.021482,1.014948,...,17.175470,-0.156228,2.749415,2.856054,12.326514,1.272234,14.983299,15.802401,6.103758,9.526305
1,P49913,0,9606,134,4.367090,0.887015,1.068507,1.090373,0.998955,0.990000,...,17.045000,-0.152060,2.848507,2.941045,12.744030,1.287090,15.187239,14.644030,6.095075,9.676866
2,P03045,0,10710,1,4.520000,1.180000,2.670000,2.960000,0.880000,1.120000,...,18.490000,-0.107000,3.630000,3.910000,15.700000,1.400000,16.250000,1.430000,5.740000,14.900000
3,P00004,0,9796,1,4.520000,1.180000,2.670000,2.960000,0.880000,1.120000,...,18.490000,-0.107000,3.630000,3.910000,15.700000,1.400000,16.250000,1.430000,5.740000,14.900000
4,P27695,0,9606,193,4.360000,0.944352,1.061969,1.103782,1.015596,1.009275,...,17.097824,-0.138544,2.995026,3.021554,13.261658,1.345751,15.096010,13.535596,6.144145,10.127979


In [9]:
prepared_data.to_csv('basic_aa_features.csv', index=False)

In [10]:
import statsmodels.api as sm

# TODO: Run linear models to see if any of the features are important
# Then run a linear model on the important ones to see how predictive that is overall
# prepared_data = pd.read_csv('basic_aa_features.csv', header=0)

# Run models on each feature:
for feat in aa_index_feats:
    feat = feat + '_avg'
    prepared_data[feat] = prepared_data[feat].astype(float)
    prepared_data[feat] = sm.add_constant(prepared_data[feat])
    model = sm.OLS(prepared_data['is_disordered'], prepared_data[feat])
    results = model.fit()
    print(feat, results.summary())

ValueError: Columns must be same length as key